In [1]:
#%%capture
print('Starting...')
import logging
from datetime import datetime
import sys
import os
import torch
major_version, minor_version = torch.cuda.get_device_capability()
# Must install separately since Colab has torch 2.2.1, which breaks packages
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
if major_version >= 8:
    # Use this for new GPUs like Ampere, Hopper GPUs (RTX 30xx, RTX 40xx, A100, H100, L40)
    !pip install --no-deps packaging ninja einops flash-attn xformers trl peft accelerate bitsandbytes
else:
    # Use this for older GPUs (V100, Tesla T4, RTX 20xx)
    print('Installing dependencies older GPUs...')
    !pip install --no-deps xformers trl peft accelerate bitsandbytes
pass

from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
import json
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, GemmaTokenizer,  StoppingCriteria, StoppingCriteriaList, GenerationConfig
from unsloth import FastLanguageModel
print('End of pip install...')

Starting...
  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-7jljqe7v/unsloth_e3b57922e6654552b91e98f7eff80f36
  Running command git clone -q https://github.com/unslothai/unsloth.git /tmp/pip-install-7jljqe7v/unsloth_e3b57922e6654552b91e98f7eff80f36
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 1.3 MB 19.6 MB/s eta 0:00:01
     |████████████████████████████████| 1.0 MB 95.1 MB/s eta 0:00:01
  Using cached wheel-0.43.0-py3-none-any.whl (65 kB)
  Created wheel for unsloth: filename=unsloth-2024.4-py3-none-any.whl size=96280 sha256=34f03863003c07402bd1019d891e3008b6816829a8065d1a0d4ee3a0e13cafae
  Stored in directory: /tmp/pip-ephem-wheel-cache-8fd_sx9d/wheels/0b/bf/f5/61523189908a01bce8752a181f02f8b057ffc2c792447d39ff
Successfully built unsloth
  Attempting uninstall: wheel
    Found existing installation:

In [2]:
print('Starting...')
# Obtener la fecha y hora actual
fecha_hora_actual = datetime.now()

# Crear un manejador para la consola
console_handler = logging.StreamHandler()
console_handler.setLevel(logging.INFO)

# Formatear la fecha y hora
texto_fecha_hora = fecha_hora_actual.strftime("%Y%m%d_%H%M")
mylog = "comebien_"+texto_fecha_hora+".log"

# Crear un manejador para el archivo
file_handler = logging.FileHandler(filename=mylog, encoding = "UTF-8")
file_handler.setLevel(logging.INFO)

# Definir el formato de los registros
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
#console_handler.setFormatter(formatter)
file_handler.setFormatter(formatter)

# Crear un logger
logger = logging.getLogger()
logger.setLevel(logging.INFO)

# Agregar los manejadores al logger
logger.addHandler(console_handler)
logger.addHandler(file_handler)
logger.info('Comienzo de script')
#from huggingface_hub import notebook_login
#notebook_login()

Comienzo de script


Starting...


In [3]:
logger.info('Cargando modelos...')
#my_run = "mistral7biv02_2""
#my_run = "gemma-2b-it-bnb-4bit"
# 23/03/24 my_run = "mistral-7b-instruct-v0.2-bnb-4bit" 
# 24/03/2024 Error con my_run = "gemma-7b-it-bnb-4bit"
# 28/03/2024 Ok 60 steps "llama-2-7b-bnb-4bit"
# 28/03/24 ok 60 "mistral-7b-instruct-v0.2-bnb-4bit"
# 29/03/24 ok 2000 1h "gemma-2b-it-bnb-4bit"
#my_run = "gemma-2b-it-bnb-4bit"
enable_gemma = True
if enable_gemma:
    my_run = 'gemma-2b-bnb-4bit'
    #sft_model_name = 'somosnlp/ComeBien_DIXREC10_gemma-2b-it-bnb-4bit'
    sft_model_name = 'somosnlp/ComeBien_Nutr100_gemma-2b-bnb-4bit'
    sft_model_name = 'somosnlp/RecetasDeLaAbuela5k_gemma-2b-bnb-4bit'
else:
    my_run = 'mistral-7b-bnb-4bit'
    sft_model_name = 'somosnlp/ComeBien_DIXREC10_mistral-7b-bnb-4bit'

base_model_name = "unsloth/" + my_run

#sft_model = "somosnlp/ComeBien_mistral-7b-instruct-v0.2-bnb-4bit"
#base_model_name = "unsloth/Mistral-7B-Instruct-v0.2"
#sft_model_name = "somosnlp/ComeBien_Ing_gemma-2b-it-bnb-4bit"
#sft_model_name = "somosnlp/ComeBien_gemma-2b-it-bnb-4bit"
#sft_model_name = "somosnlp/ComeBien_URL10_gemma-2b-it-bnb-4bit"

logger.info('Usando modelo base '+base_model_name)
logger.info('Usando modelo SFT '+sft_model_name)

max_seq_length=150
#base_model = AutoModelForCausalLM.from_pretrained(base_model_name,return_dict=True,device_map="auto",torch_dtype=torch.float16,)
#tokenizer = AutoTokenizer.from_pretrained(base_model_name, max_length = max_seq_length)

if True:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(model_name = sft_model_name,max_seq_length = max_seq_length,dtype=None, load_in_4bit = True,)

if False:
    from unsloth import FastLlamaModel, FastMistralModel
    base_model, tokenizer = FastMistralModel.from_pretrained(model_name = base_model_name,max_seq_length = max_seq_length,dtype = None,load_in_4bit = True,)
    logger.info('Modelo base cargado ' +str(base_model_name))
    ft_model = PeftModel.from_pretrained(base_model, sft_model_name, torch_dtype=torch.float16,)
    logger.info(type(ft_model))
    logger.info('Mezclando modelos...')
    model = ft_model.merge_and_unload()

logger.info(type(model))
model.save_pretrained("./model")
tokenizer.save_pretrained("./model")
logger.info('Modelo cargado ' +str(sft_model_name))

Cargando modelos...
Usando modelo base unsloth/gemma-2b-bnb-4bit
Usando modelo SFT somosnlp/RecetasDeLaAbuela5k_gemma-2b-bnb-4bit


adapter_config.json:   0%|          | 0.00/730 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Gemma patching release 2024.4
   \\   /|    GPU: Tesla T4. Max memory: 14.581 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.2+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.25.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


model.safetensors:   0%|          | 0.00/2.07G [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

PyTorch version 2.2.2 available.


adapter_config.json:   0%|          | 0.00/730 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/78.5M [00:00<?, ?B/s]

Unsloth 2024.4 patched 18 layers with 18 QKV layers, 18 O layers and 18 MLP layers.
<class 'peft.peft_model.PeftModelForCausalLM'>
Modelo cargado somosnlp/RecetasDeLaAbuela5k_gemma-2b-bnb-4bit


<a name="Inference"></a>
### Inference
Let's run the model! You can change the instruction and input - leave the output blank!

In [4]:
# Inference functions
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    #instructions = examples["instruction"]
    inputs       = examples["Nombre"]
    outputs      = examples["DIXREC"]
    texts = []
    for input, output in zip(inputs, outputs):
        instruction = 'Eres un agente experto en nutrición y cocina.'
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass


# alpaca_prompt = Copied from above
def generar_alpaca_respuesta(pregunta, contexto, logger, model):
    prompt = alpaca_prompt.format(contexto, pregunta, '',) + EOS_TOKEN
    logger.info(prompt)
    inputs = tokenizer([prompt], return_tensors = "pt").to("cuda")    
    outputs = model.generate(**inputs, max_new_tokens = 100, use_cache = True)    
    res=tokenizer.batch_decode(outputs)    
    logger.info(res)
    return res


class ListOfTokensStoppingCriteria(StoppingCriteria):
    """
    Clase para definir un criterio de parada basado en una lista de tokens específicos.
    """
    def __init__(self, tokenizer, stop_tokens):
        self.tokenizer = tokenizer
        # Codifica cada token de parada y guarda sus IDs en una lista
        self.stop_token_ids_list = [tokenizer.encode(stop_token, add_special_tokens=False) for stop_token in stop_tokens]

    def __call__(self, input_ids, scores, **kwargs):
        # Verifica si los últimos tokens generados coinciden con alguno de los conjuntos de tokens de parada
        for stop_token_ids in self.stop_token_ids_list:
            len_stop_tokens = len(stop_token_ids)
            if len(input_ids[0]) >= len_stop_tokens:
                if input_ids[0, -len_stop_tokens:].tolist() == stop_token_ids:
                    return True
        return False

# Uso del criterio de parada personalizado
stop_tokens = ["<end_of_turn>"]  # Lista de tokens de parada

# Inicializa tu criterio de parada con el tokenizer y la lista de tokens de parada
stopping_criteria = ListOfTokensStoppingCriteria(tokenizer, stop_tokens)

# Añade tu criterio de parada a una StoppingCriteriaList
stopping_criteria_list = StoppingCriteriaList([stopping_criteria])

def generate_text(prompt, context, model, max_length=max_seq_length):
  prompt=prompt.replace("\n", "").replace("¿","").replace("?","")
  #input_text = f'''<bos><start_of_turn>system ¿{context}?<end_of_turn><start_of_turn>user ¿{prompt}?<end_of_turn><start_of_turn>model'''
  #https://unsloth.ai/blog/gemma-bugs
  enable_gemma = True
  if enable_gemma:
    input_text = f'''<bos><start_of_turn>system\n{context}<end_of_turn><start_of_turn>user\n{prompt}<end_of_turn><start_of_turn>model\n'''
  else:
    intro = 'Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request'
    input_text = f'''{intro}\n\n### Instruction:\n\n{context}\n\n### Input:\n{prompt}\n\n### Response: '''


  inputs = tokenizer.encode(input_text, return_tensors="pt", add_special_tokens=False).to("cuda:0")
  max_new_tokens=max_length
  generation_config = GenerationConfig(max_new_tokens=max_new_tokens,
                temperature=0.32, #top_p=0.9, top_k=50, # 45
                repetition_penalty=1.3,  #1.1 #1.04
                do_sample=True,
            )
  if enable_gemma:
    outputs = model.generate(generation_config=generation_config, input_ids=inputs, stopping_criteria=stopping_criteria_list,)
  else:
    outputs = model.generate(generation_config=generation_config, input_ids=inputs)
  return tokenizer.decode(outputs[0], skip_special_tokens=False) #True

def mostrar_respuesta(pregunta, contexto, logger, model):
    try:
        res= generate_text(pregunta, contexto, model, 700)
        print(res)
    except Exception as e:
        res = e    
    #logger.info(res.encode('utf-8'))
    logger.info(str(res))
    return res


In [5]:
 torch.cuda.is_bf16_supported()

False

In [6]:
test1 = False
if test1:
    context = 'Eres un agente experto en nutrición y cocina'     
    instruction = 'Cantidad de calorías grasas sodio fibra sal azúcar de '
    items = ['Tortilla de patatas', 'Ceviche', 'Enchiladas']
    for item in items:
        pregunta = instruction + item
        mostrar_respuesta(pregunta, context, logger, model)

In [7]:
test2 = False
if test2:
    context = "You are a technical specialist in food and cooking. Dime en español el valor nutricional de "
    mostrar_respuesta("Tortilla de patatas", context, logger, model)
    mostrar_respuesta("Ceviche", context, logger, model)
    mostrar_respuesta("Enchiladas", context, logger, model)
    context = "You are a technical specialist in food and cooking. Resume en una sola frase el valor nutricional de "
    mostrar_respuesta("Tortilla de patatas", context, logger, model)
    mostrar_respuesta("Ceviche", context, logger, model)
    mostrar_respuesta("Enchiladas", context, logger, model)

In [ ]:
!pip install evaluate bert_score
from evaluate import load
bertscore = load("bertscore")

In [51]:
import pandas as pd

# Paso 1: Leer el archivo CSV en un DataFrame
archivo_csv = 'recetasabuela_es.csv'  # Reemplaza 'nombre_del_archivo.csv' con el nombre de tu archivo CSV
dft = pd.read_csv(archivo_csv)

test30 = True
if test30:
    context = "You are a technical specialist in food and cooking. Dime los ingredientes de "
    context = "Eres un agente experto en nutrición y cocina."
    query = 'Ingredientes y pasos de la receta '    
    #recetas = ["Tortilla de patatas", "Ceviche", "Enchiladas", "Empanada de puerro y nata", "Cordero asado", "Paella", "Guiso de carne", "Potaje de garbanzos", "Callos madrileños" ]
    #recetas = ["repollo guisado"]
    #for receta in recetas:
    df = dft.head(20)
    for indice, fila in df.iterrows():        
        receta = query + fila['Nombre']        
        output = mostrar_respuesta(receta, context, logger, model)
        predictions = [receta, output]
        ref_output = 'Ingredientes: '+fila['Ingredientes']+'. Pasos: '+fila['Pasos']
        references = [receta, ref_output]
        results = bertscore.compute(predictions=predictions, references=references, lang="es")
        logger.info(results)
        df.loc[indice, 'precision'] = results['precision'][1]
        df.loc[indice, 'recall'] = results['recall'][1]
        df.loc[indice, 'f1'] = results['f1'][1]
    df.to_csv('recetasabuela_es_ds.csv', index=False)
    logger.info('Promedio precision: '+str(df['precision'].mean()))
    logger.info('Promedio recall: '+str(df['recall'].mean()))
    logger.info('Promedio f1: '+str(df['f1'].mean()))

<bos><start_of_turn>system
Eres un agente experto en nutrición y cocina.<end_of_turn><start_of_turn>user
Ingredientes y pasos de la receta tarta de pollo y verduras<end_of_turn><start_of_turn>model
Ingredientes: 1 disco de masa para tarta (aproximadamente una lámina cuadrada) ½ kilogramo de pepino rojo o verde ¼ pieza pequeña de cebolla morada ¾ kilogramos de calabaza 20 gramos de harina integral 3 cucharadas soperas de aceite 1 cucharada postre de sal 1 huevo cocido 1 pizca de pimienta negra molida 4 tazas de caldo vegetal caliente. Pasos: 5 Precalentar el horno a 180 ºC con calor arriba y abajo. Para hacer esta tarta de carne picada puedes usar lo que hayens ha llamado "carne molida" es decir unas tiras pequeñas del tamaño deseado pero sin cortarlas por completo así como también se hace cuando uno elabora su propio chorizo casero siguiendo las instrucciones paso bpaso pasoblp. Truco: Si no tienes este tipo de corteza le vas a servir más tiempo al relleno porque tiene menos grasa. 6 P

<bos><start_of_turn>system
Eres un agente experto en nutrición y cocina.<end_of_turn><start_of_turn>user
Ingredientes y pasos de la receta tarta de pollo y verduras<end_of_turn><start_of_turn>model
Ingredientes: 1 disco de masa para tarta (aproximadamente una lámina cuadrada) ½ kilogramo de pepino rojo o verde ¼ pieza pequeña de cebolla morada ¾ kilogramos de calabaza 20 gramos de harina integral 3 cucharadas soperas de aceite 1 cucharada postre de sal 1 huevo cocido 1 pizca de pimienta negra molida 4 tazas de caldo vegetal caliente. Pasos: 5 Precalentar el horno a 180 ºC con calor arriba y abajo. Para hacer esta tarta de carne picada puedes usar lo que hayens ha llamado "carne molida" es decir unas tiras pequeñas del tamaño deseado pero sin cortarlas por completo así como también se hace cuando uno elabora su propio chorizo casero siguiendo las instrucciones paso bpaso pasoblp. Truco: Si no tienes este tipo de corteza le vas a servir más tiempo al relleno porque tiene menos grasa. 6 P

/tmp/ipykernel_50/3372286814.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[indice, 'precision'] = results['precision'][1]
/tmp/ipykernel_50/3372286814.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[indice, 'recall'] = results['recall'][1]
/tmp/ipykernel_50/3372286814.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.o

<bos><start_of_turn>system
Eres un agente experto en nutrición y cocina.<end_of_turn><start_of_turn>user
Ingredientes y pasos de la receta tarta de coliflor y salsa blanca<end_of_turn><start_of_turn>model
Ingredientes: 1 Coliflor grande (el doble que el recipiente donde se va a preparar) ½ Cebolla mediana ¼ Pimiento rojo mediano 2 dientes de Ajo picado muy fino Sal Pimienta Aceite para freír. Pasos: 1 Para empezar con esta preparación corta toda las verduras finamente al igual que los dientes de ajo picados pero más pequeños ya que es lo primero que haremos antes de ir agregando aceite o mantequilla en una sartén amplia hasta cubrir todo bien por encima del fondo dejándola doradita durante unos minutos aproximadamente así no quedará tan líquida como si estuviéramos cocinándolas sin sofreirlas previamente. Truco: Si quieres hacer tu propia sal también puedes hacerlo tú mismo siguiendo nuestra Receta fácil de sal gorda casera ¡te encantará! 2 Luego agrega poco a poco agua caliente mientr

<bos><start_of_turn>system
Eres un agente experto en nutrición y cocina.<end_of_turn><start_of_turn>user
Ingredientes y pasos de la receta tarta de pollo y champinones<end_of_turn><start_of_turn>model
Ingredientes: 1 disco para tarta o masa sin gluten (yo he utilizado una masa que tiene harina integral) ½ kilogramo de pechuga de Pollo cocida ¼ unidad de Cebolla blanca mediana 2 unidades de Champiñones frescos laminados ¾ taza de Queso parmesano rallado Sal Pimienta negra molida Aceite de oliva virgen extra Vinagre blanco al gusto. Pasos: 0 Para empezar a preparar esta deliciosa tarta casera primero debes reunir todos los ingredientes. Truco: Si no consigues discos especiales como este puedes usar cualquier tipo de lámina plana tal y como lo hacemos con las sartenes koncentrantes por ejemplo. 1 A continuación corta el queso fresco en cubos pequeños pela la cebolla y lávala bien bajo agua fría así quedarán más líquidas cuando se triture todo junto durante unos minutos. Luego sofríe liger

...

<bos><start_of_turn>system
Eres un agente experto en nutrición y cocina.<end_of_turn><start_of_turn>user
Ingredientes y pasos de la receta sandwich de pollo desmenuzado<end_of_turn><start_of_turn>model
Ingredientes: 2 rebanadas de Pan integral o pan blanco ½ unidad de Pechuga de Pollo cocida ¼ taza de Mayonesa (60 mililitros) 1 cucharada sopera de Salsa inglesa Sal Pimienta al gusto. Pasos: 3 Preparamos todos los ingredientes para empezar a preparar el sándwich de pechuga de pollo desmeudado. Truco: Puedes utilizar mayonesa casera si quieres hacerla tu mismo ¡es muy fácil! 4 En primer lugar cortamos las rodajas del pan por medio aproximadamente como se ve en la foto. Luego colocamos una buena porción de salsa inglesa sobre cada mitad de pan. 5 A continuación añadimos encima unas cuantas cucharadas de mightonaise que hemos puesto previamente salpimentado al gusto. Después espolvoreamos con suficiente pimienta negra recién molida. Si lo prefieres puedes usar otra especie de mostaza pero 

<bos><start_of_turn>system
Eres un agente experto en nutrición y cocina.<end_of_turn><start_of_turn>user
Ingredientes y pasos de la receta sandwich de pechuga de pollo con tomate<end_of_turn><start_of_turn>model
Ingredientes: 20 rebanadas de Pan integral (también se puede usar pan normal) ½ unidad de Pechuga de Pollo cocida ¼ pieza de Tomate natural frito 1 pizca de Sal Aceite virgen extra Salsa inglesa para cocinar o Mayonesa. Pasos: 3 Alistar todos los ingredientes que vamos a utilizar en el momento del plato es muy importante porque esto le da mayor orden al proceso. Truco: Si no tienes mayonesa puedes mezclar salsa Inglés con aceite de oliva sal y pimienta negra molida. 4 Para empezar colocamos las dos piezas de pan una encima de otra como si fuera una tapa pero sin cerrarlas por completo. Así tendremos nuestro sándwich abierto listo para rellenar. 5 Luego ponemos unas cucharadas grandes de tomatero sobre cada lado de nuestros panes tal como ves en la imagen. El tamaño depende much

<bos><start_of_turn>system
Eres un agente experto en nutrición y cocina.<end_of_turn><start_of_turn>user
Ingredientes y pasos de la receta sandwich de pechuga de pollo con tomate<end_of_turn><start_of_turn>model
Ingredientes: 20 rebanadas de Pan integral (también se puede usar pan normal) ½ unidad de Pechuga de Pollo cocida ¼ pieza de Tomate natural frito 1 pizca de Sal Aceite virgen extra Salsa inglesa para cocinar o Mayonesa. Pasos: 3 Alistar todos los ingredientes que vamos a utilizar en el momento del plato es muy importante porque esto le da mayor orden al proceso. Truco: Si no tienes mayonesa puedes mezclar salsa Inglés con aceite de oliva sal y pimienta negra molida. 4 Para empezar colocamos las dos piezas de pan una encima de otra como si fuera una tapa pero sin cerrarlas por completo. Así tendremos nuestro sándwich abierto listo para rellenar. 5 Luego ponemos unas cucharadas grandes de tomatero sobre cada lado de nuestros panes tal como ves en la imagen. El tamaño depende much

<bos><start_of_turn>system
Eres un agente experto en nutrición y cocina.<end_of_turn><start_of_turn>user
Ingredientes y pasos de la receta quiche de tomates cherry y queso<end_of_turn><start_of_turn>model
Ingredientes: 1 lámina para quiche (25 cm) ½ taza de leche entera o vegetal ¾ cucharada postre de nuez moscada ¼ cucharadita de pimienta negra molida 1½ tazas pequeñas de tomate Cherry picada finamente al gusto Queso rallado tipo mozzarella picado a cuadros Sal Ginkgo biloba Aceite virgen extra. Pasos: 1 Para empezar con esta deliciosa quiche casera primero prepara el relleno del guiso de carne fácil haciendo una salsa bechamel muy suave añadiendo poco por poco las dos cucharadas soperas de leche junto con media pizca de sal ginkgo y aceite. Luego agrega los ingredientes restantes hasta que se integren bien todos juntos como se ve en la foto. Truco: Puedes utilizar cualquier otro ingrediente si lo prefieres ¡esta es tu propia versión! 2 A continuación coloca este preparado dentro de l

<bos><start_of_turn>system
Eres un agente experto en nutrición y cocina.<end_of_turn><start_of_turn>user
Ingredientes y pasos de la receta quiche de tomates cherry y queso<end_of_turn><start_of_turn>model
Ingredientes: 1 lámina para quiche (25 cm) ½ taza de leche entera o vegetal ¾ cucharada postre de nuez moscada ¼ cucharadita de pimienta negra molida 1½ tazas pequeñas de tomate Cherry picada finamente al gusto Queso rallado tipo mozzarella picado a cuadros Sal Ginkgo biloba Aceite virgen extra. Pasos: 1 Para empezar con esta deliciosa quiche casera primero prepara el relleno del guiso de carne fácil haciendo una salsa bechamel muy suave añadiendo poco por poco las dos cucharadas soperas de leche junto con media pizca de sal ginkgo y aceite. Luego agrega los ingredientes restantes hasta que se integren bien todos juntos como se ve en la foto. Truco: Puedes utilizar cualquier otro ingrediente si lo prefieres ¡esta es tu propia versión! 2 A continuación coloca este preparado dentro de l

In [52]:
logger.info('Promedio precision: '+str(df['precision'].mean()))
logger.info('Promedio recall: '+str(df['recall'].mean()))
logger.info('Promedio f1: '+str(df['f1'].mean()))

Promedio precision: 0.6740117996931076
Promedio recall: 0.717666494846344
Promedio f1: 0.6951073169708252


In [11]:
test31 = False
if test31:
    context = "You are a technical specialist in food and cooking. Dime los ingredientes de "
    context = "Eres un agente experto en nutrición y cocina."
    query = 'Dime los ingredientes y pasos de la receta '
    recetas = ["Tortilla de patatas", "Ceviche", "Enchiladas", "Empanada de puerro y nata", "Cordero asado", "Paella", "Guiso de carne", "Potaje de garbanzos", "Callos madrileños" ]
    for receta in recetas:
        receta = query + receta
        mostrar_respuesta(receta, context, logger, model)
    
    #mostrar_respuesta("¡Un delicioso milhojas de otoño!", context, logger, model)
    #mostrar_respuesta("milhojas de otoño", context, logger, model)
    #mostrar_respuesta("¡No has probado nada igual!", context, logger, model)
    #mostrar_respuesta("¡Un plato rápido y saludable!", context, logger, model)

In [ ]:
test32 = True
if test32:
    context = "You are a technical specialist in food and cooking. Dime los ingredientes de "
    context = "Eres un agente experto en nutrición y cocina."
    query = 'Dime los ingredientes y pasos de la receta '
    recetas = ["Pollo en pepitoria", "Lombarda con piñones", "Huevos a la flamenca", "Guisantes con jamón", "Empanada de carne", "Rosbif de ternera", "Cochinillo", "Puré de verduras" ]
    for receta in recetas:
        receta = query + receta
        mostrar_respuesta(receta, context, logger, model)
    

In [13]:
test33 = True
if test33:
    context = "You are a technical specialist in food and cooking. Dime los ingredientes de "
    context = "Eres un agente experto en nutrición y cocina."
    query = 'Dime los ingredientes y pasos de la receta '
    recetas = ["garbanzos criollos", "mariscada al ajillo"]
    for receta in recetas:
        receta = query + receta
        mostrar_respuesta(receta, context, logger, model)

<bos><start_of_turn>system
Eres un agente experto en nutrición y cocina.<end_of_turn><start_of_turn>user
Dime los ingredientes y pasos de la receta garbanzos criollos<end_of_turn><start_of_turn>model
Ingredientes: 150 gramos de Garbanzos cocidos (o frescos) ½ unidad de Cebolla blanca ¼ taza de Tomate triturado natural o frito 2 unidades de Zanahorias medianas peladas 1 pizca de Sal al gusto 1 chorro de Aceite de oliva virgen extra para cocinar 1 cucharadita de Comino molido 1 cucharada sopera de Perejil fresco picado seco 1 diente de Ajo machacado en el mortero 1 hoja de Laurel 1 rama de Romero 1 copa pequeña de Vino blanco seco. Pasos: 1 Para empezar a preparar estos deliciosos platos peruanos con fideos lo primero que debes hacer es hervir las legumbres durante una hora aproximadamente procurando siempre conservar su punto de cocción deseado. Una vez cocinados escúrrelos bien escurren bajo agua fría reservando medio litro del líquido donde se han lavado así podrás utilizarlo como cal

<bos><start_of_turn>system
Eres un agente experto en nutrición y cocina.<end_of_turn><start_of_turn>user
Dime los ingredientes y pasos de la receta garbanzos criollos<end_of_turn><start_of_turn>model
Ingredientes: 150 gramos de Garbanzos cocidos (o frescos) ½ unidad de Cebolla blanca ¼ taza de Tomate triturado natural o frito 2 unidades de Zanahorias medianas peladas 1 pizca de Sal al gusto 1 chorro de Aceite de oliva virgen extra para cocinar 1 cucharadita de Comino molido 1 cucharada sopera de Perejil fresco picado seco 1 diente de Ajo machacado en el mortero 1 hoja de Laurel 1 rama de Romero 1 copa pequeña de Vino blanco seco. Pasos: 1 Para empezar a preparar estos deliciosos platos peruanos con fideos lo primero que debes hacer es hervir las legumbres durante una hora aproximadamente procurando siempre conservar su punto de cocción deseado. Una vez cocinados escúrrelos bien escurren bajo agua fría reservando medio litro del líquido donde se han lavado así podrás utilizarlo como cal

<bos><start_of_turn>system
Eres un agente experto en nutrición y cocina.<end_of_turn><start_of_turn>user
Dime los ingredientes y pasos de la receta mariscada al ajillo<end_of_turn><start_of_turn>model
Ingredientes: 1 kilogramo de Mariscos (calamar limpio gambas mejillones langostinos o mixto) 2 dientes de Ajo ½ vaso pequeño de Aceite de oliva virgen extra 1 pizca de Sal. Pasos: 0 Para empezar con esta deliciosa preparación para Navidad primero prepara todos los mariscos que vas a utilizar. Si lo prefieres puedes comprarlos ya listos pero nosotros hemos optado por hacerlo todo mismo así como hacer una picada del ajo junto con el aceite que vamos a usar durante toda la cocción del marisco. Truco: Puedes agregar más tipos de moluscas si quieres aunque no son obligatorios. 1 Ahora preparamos nuestra salsa base. Entonces corta finamente este pimiento rojo bien trocitos pequeños e incorpora esto también a tu mortero donde has picado previamente tanto el ajo como el aceite. Mezcla muy bien ha

<bos><start_of_turn>system
Eres un agente experto en nutrición y cocina.<end_of_turn><start_of_turn>user
Dime los ingredientes y pasos de la receta mariscada al ajillo<end_of_turn><start_of_turn>model
Ingredientes: 1 kilogramo de Mariscos (calamar limpio gambas mejillones langostinos o mixto) 2 dientes de Ajo ½ vaso pequeño de Aceite de oliva virgen extra 1 pizca de Sal. Pasos: 0 Para empezar con esta deliciosa preparación para Navidad primero prepara todos los mariscos que vas a utilizar. Si lo prefieres puedes comprarlos ya listos pero nosotros hemos optado por hacerlo todo mismo así como hacer una picada del ajo junto con el aceite que vamos a usar durante toda la cocción del marisco. Truco: Puedes agregar más tipos de moluscas si quieres aunque no son obligatorios. 1 Ahora preparamos nuestra salsa base. Entonces corta finamente este pimiento rojo bien trocitos pequeños e incorpora esto también a tu mortero donde has picado previamente tanto el ajo como el aceite. Mezcla muy bien ha

In [14]:
test34 = True
if test34:
    context = "Eres un agente experto en nutrición y cocina."
    #query = 'Dime los nombres de las recetas más populares que conozcas similares a '
    query = 'Dime el valor nutricional de '
    recetas = ["arrroz con pollo", "carne de cerdo guisada"]
    for receta in recetas:
        receta = query + receta
        mostrar_respuesta(receta, context, logger, model)

<bos><start_of_turn>system
Eres un agente experto en nutrición y cocina.<end_of_turn><start_of_turn>user
Dime el valor nutricional de arrroz con pollo<end_of_turn><start_of_turn>model
Ingredientes e pasos de la receta arroz caldoso para ninos kasama pollo al microondas<unused74>~author
Ingredientes: 1 taza de Arroz cocido ½ unidad de Pechuga de Pollo cocinada ¼ lata de Maíz dulce (20 g) ½ pieza de Cebolla blanca cortada finamente 4 piezas de Patata mediana picada a cuadritos Sal gruesa o molida Pimienta negra recién sazonada Tomate frito en cubo Queso parmesano rallado. Pasos: 3 minutos antes del tiempo máximo de cocción se coloca una sartén grande junto con las patatas picadas previamente salpimentadas. Se rehoga todo hasta que esté bien hecho pero no muy tierno ya que luego le dará calor al caldo por lo tanto es importante que los ingredientes queden tiernos tal como si fueran papas asadas. Truco: Si quieres hacer tu propio arroz puedes seguir esta sencilla receta paso b passo paso ¿

<bos><start_of_turn>system
Eres un agente experto en nutrición y cocina.<end_of_turn><start_of_turn>user
Dime el valor nutricional de arrroz con pollo<end_of_turn><start_of_turn>model
Ingredientes e pasos de la receta arroz caldoso para ninos kasama pollo al microondas<unused74>~author
Ingredientes: 1 taza de Arroz cocido ½ unidad de Pechuga de Pollo cocinada ¼ lata de Maíz dulce (20 g) ½ pieza de Cebolla blanca cortada finamente 4 piezas de Patata mediana picada a cuadritos Sal gruesa o molida Pimienta negra recién sazonada Tomate frito en cubo Queso parmesano rallado. Pasos: 3 minutos antes del tiempo máximo de cocción se coloca una sartén grande junto con las patatas picadas previamente salpimentadas. Se rehoga todo hasta que esté bien hecho pero no muy tierno ya que luego le dará calor al caldo por lo tanto es importante que los ingredientes queden tiernos tal como si fueran papas asadas. Truco: Si quieres hacer tu propio arroz puedes seguir esta sencilla receta paso b passo paso ¿

<bos><start_of_turn>system
Eres un agente experto en nutrición y cocina.<end_of_turn><start_of_turn>user
Dime el valor nutricional de carne de cerdo guisada<end_of_turn><start_of_turn>model
Ingredientes e pasos de la receta cazuela con verduras al hornocauthor
Ingredientes: 1 kilogramo de Carne de cerdo para guisar (de unos dos kilos) ½ unidad de Cebolla blanca mediana ¼ taza de Tomate triturado natural o frito 2 unidades de Zanahorias medianas troceadas porciones pequeñas del tamaño que queramos de otros vegetales como calabaza puerro cebolla morrón pimiento etc. ½ vaso pequeño de Vino blanco seco Aceite Sal Pimienta negra molida Perejil picado Jengibre fresco Corta todo lo vegetal finamente picado exceptuando las zanahorias que se dejan enteras ¡y a cocinar! Pasos: 01 Precalentar el horno a 185 ºC-200 ºC durante diez minutos antes de emplatar esta deliciosa salsa verde casera. Truco: Puedes agregar más ingredientes si te gusta este tipo de recetas fáciles pero no demasiado porque en

<bos><start_of_turn>system
Eres un agente experto en nutrición y cocina.<end_of_turn><start_of_turn>user
Dime el valor nutricional de carne de cerdo guisada<end_of_turn><start_of_turn>model
Ingredientes e pasos de la receta cazuela con verduras al hornocauthor
Ingredientes: 1 kilogramo de Carne de cerdo para guisar (de unos dos kilos) ½ unidad de Cebolla blanca mediana ¼ taza de Tomate triturado natural o frito 2 unidades de Zanahorias medianas troceadas porciones pequeñas del tamaño que queramos de otros vegetales como calabaza puerro cebolla morrón pimiento etc. ½ vaso pequeño de Vino blanco seco Aceite Sal Pimienta negra molida Perejil picado Jengibre fresco Corta todo lo vegetal finamente picado exceptuando las zanahorias que se dejan enteras ¡y a cocinar! Pasos: 01 Precalentar el horno a 185 ºC-200 ºC durante diez minutos antes de emplatar esta deliciosa salsa verde casera. Truco: Puedes agregar más ingredientes si te gusta este tipo de recetas fáciles pero no demasiado porque en

In [15]:
test4 = False
if test4:
    context = "Eres un especialista en nutricion y cocina. Dime el valor nutricional de la "
    context = 'Eres un agente experto en nutrición y cocina. Cantidad de calorías, grasas, sodio, azúcar'
    #context = 'You are a helpful AI assistant. Eres un agente experto en nutrición y cocina. Lista de codigos linguisticos disponibles: [''es'', ''en'']'
    #mostrar_respuesta(" Dime la cantidad de grasas de la capirotada", context, logger)
    mostrar_respuesta("capirotada", context, logger, model)
    context = 'Eres un agente experto en nutrición y cocina. Cantidad de calorías, grasas, sodio, azúcar'
    #context = 'You are a helpful AI assistant. Eres un agente experto en nutrición y cocina. Lista de codigos linguisticos disponibles: [''es'', ''en'']'
    #mostrar_respuesta(" Dime la cantidad de grasas de la capirotada", context, logger)
    mostrar_respuesta("capirotada", context, logger, model)
    context = 'Eres un agente experto en nutrición y cocina'
    mostrar_respuesta("Dime la receta de la capirotada", context, logger, model)
    mostrar_respuesta("Dime la receta de los Tacos Dorados de Papa", context, logger, model)
    mostrar_respuesta("Dime la receta de Coditos con queso Amarillo y pimiento morrón", context, logger, model)
    mostrar_respuesta("Dime la receta de lomo saltado con tallarines a la huancai", context, logger, model)
    mostrar_respuesta("Dime la duración de la receta Cebollas Moradas Curtidas", context, logger, model)
    mostrar_respuesta("Dime la duración de la receta Tortilla de patatas Cebollas Moradas Curtidas", context, logger, model)

In [16]:
test5 = False
if test5:
    context = 'Eres un agente experto en nutrición y cocina'
    instruction = 'Cantidad de calorías, grasas, sodio, azúcar de la receta '
    test = ['Tacos Dorados de Papa','Ensalada de Chayotes', 'Huevos a la Mexicana', 'Rajas de Chile Poblano con Crema', 'Papas a la Mexicana','Nopalitos navegantes', 'Guacamole Casero', 'Calabacitas al vapor','Sopa de Calabacitas con Elote','Pasta con salsa cremosa de chile poblano']
    for item in test:
        item = instruction + item
        logger.info('Lanzando test ' + str(item))
        mostrar_respuesta(item, context, logger, model)

In [17]:
test6 = False
if test6:
    context = 'Eres un agente experto en nutrición y cocina'
    instruction = 'País originario de la receta '
    test = ['Tacos Dorados de Papa','Ensalada de Chayotes', 'Huevos a la Mexicana', 'Rajas de Chile Poblano con Crema', 'Papas a la Mexicana','Nopalitos navegantes', 'Guacamole Casero', 'Calabacitas al vapor','Sopa de Calabacitas con Elote','Pasta con salsa cremosa de chile poblano']
    for item in test:
        item = instruction + item
        logger.info('Lanzando test ' + str(item))
        mostrar_respuesta(item, context, logger, model)

In [18]:
test7 = False
if test7:
    context = 'Eres un agente experto en nutrición y cocina'
    instruction = 'Tiempo de preparación de la receta '
    test = ['Tacos Dorados de Papa','Ensalada de Chayotes', 'Huevos a la Mexicana', 'Rajas de Chile Poblano con Crema', 'Papas a la Mexicana','Nopalitos navegantes', 'Guacamole Casero', 'Calabacitas al vapor','Sopa de Calabacitas con Elote','Pasta con salsa cremosa de chile poblano']
    for item in test:
        item = instruction + item
        logger.info('Lanzando test ' + str(item))
        mostrar_respuesta(item, context, logger, model)

In [19]:
test8 = False
if test8:
    context = 'Eres un agente experto en nutrición y cocina'
    instruction = 'Grado de dificultad de la receta '
    test = ['Tacos Dorados de Papa','Ensalada de Chayotes', 'Huevos a la Mexicana', 'Rajas de Chile Poblano con Crema', 'Papas a la Mexicana','Nopalitos navegantes', 'Guacamole Casero', 'Calabacitas al vapor','Sopa de Calabacitas con Elote','Pasta con salsa cremosa de chile poblano']
    for item in test:
        item = instruction + item
        logger.info('Lanzando test ' + str(item))
        mostrar_respuesta(item, context, logger, model)

In [20]:
test9 = False
if test9:
    context = 'Eres un agente experto en nutrición y cocina'
    #instruction = 'Dime la URL de la receta '
    instruction = 'Dime la DIXREC de la receta '
    test = ['Tacos Dorados de Papa','Ensalada de Chayotes', 'Huevos a la Mexicana', 'Rajas de Chile Poblano con Crema', 'Papas a la Mexicana','Nopalitos navegantes', 'Guacamole Casero', 'Calabacitas al vapor','Sopa de Calabacitas con Elote','Pasta con salsa cremosa de chile poblano']
    for item in test:
        item = instruction + item
        logger.info('Lanzando test ' + str(item))
        mostrar_respuesta(item, context, logger, model)

In [21]:
test10 = False
if test10:
    context = 'Eres un agente experto en nutrición y cocina'
    instruction = 'Dime la DIXREC de la receta '
    test = ['Tacos Dorados de Papa','Ensalada de Chayotes', 'Huevos a la Mexicana', 'Rajas de Chile Poblano con Crema', 'Papas a la Mexicana','Nopalitos navegantes', 'Guacamole Casero', 'Calabacitas al vapor','Sopa de Calabacitas con Elote','Pasta con salsa cremosa de chile poblano']
    for item in test:
        item = instruction + item
        logger.info('Lanzando test ' + str(item))
        #mostrar_respuesta(item, context, logger, model)
        generar_alpaca_respuesta(item, context, logger, model, model)

In [22]:
# alpaca_prompt = Copied from above
test11 = False
if test11: 
    #import time
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference
    instruction = 'Eres un agente experto en nutrición y cocina.'
    output = ''
    inputs_list = ['Tacos Dorados de Papa','Ensalada de Chayotes', 'Huevos a la Mexicana', 'Rajas de Chile Poblano con Crema', 'Papas a la Mexicana','Nopalitos navegantes', 'Guacamole Casero', 'Calabacitas al vapor','Sopa de Calabacitas con Elote','Pasta con salsa cremosa de chile poblano']
    input = 'Tacos Dorados de Papa'
    input = 'Rajas de Chile Poblano con Crema'

    #for input in inputs_list:
    #    print(input)
    inputs = tokenizer([alpaca_prompt.format(instruction, input, output,)], return_tensors = "pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
    tokenizer.batch_decode(outputs)
    #time.sleep(5)